# BiSRe-LLM: Perturbation Experiment with Gwen2.5-7B-Instruct

In this notebook I set up the perturbation experiments for Gwen2.5-7B-Instruct. First, the general infrastructure for interacting with Gwen2.5-7B-Instruct is set up. Second, the prompt is designed as well as the iterative API requests to Huggingface to generate the answers to the according questions and answer options. In a third and last step, the answers of the model are stored in a dataframe for further analysis.

## Dataset

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

main_question = ["question"]
main_answer = ["list_answer_options"]

#bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle", "list_answer_middle", "list_answer_options_reversed", "list_answer_options", "original_personality_prime", "original_emotional_prime"]
#non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym","question_paraphrased", "question_keyboardtypo"]

# delayed experiments with middle category
bias_answer_columns = ["list_answer_middle"]
non_bias_question_columns = []

# read the preprocessed questionnaire
#questionnaire =  pd.read_excel('../assets/20241029_selected_questionnaire.xlsx', engine='openpyxl')
questionnaire =  pd.read_csv('../assets/20241029_selected_questionnaire.csv', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["selected"]==1]
print(questionnaire.shape)
questionnaire.head(2)

(62, 20)


,Unnamed: 0,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,list_answer_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo,selected
0,0,Q1,"Important in life: Family For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Family",Q1,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ic life: Fimily nor eaca jf the following aspecto, indicete how important it ir wn your liae. Would bou say it is very imporcant, rathes importano, not very imporgant ob not impodtant at all? Famizy","Impartont in life: Flmiay For each of the following sapects, indicate how important it is in your life. uoWld you yas it is evry important, ratreh important, ton very important or not important ta all? Family","Crucial in life: Family For each of the following aspects, indicate how significant it is in your life. Would you say it is very important, rather important, not very important or not at all important? Family","How important is family to you? Please rate its significance in your life on a scale of ""very important"" to ""not important at all"".","Imporyant in kife: Family For esch of rhe following aspects, indicate how important ut is un your lide. Would you say it ia very imporrant, rather importamt, nit very important ir nor important ay all? Fanily",1
1,1,Q2,"Important in life: Friends For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Friends",Q2,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather Important ', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ix life: Fpiends For xach of the following aspects, indicate how important iv iu in kour lifev Would dou say it os very importcnt, rather impoutant, nom vwry important hr not important xt bll? Friemds","Important ni :ifel Frisnde For each of the following aspects, indicaet 

## Experiments on Remote PC
The following section contains the experimental setup and scripts conducted on the remote PC.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"

messages = [
    {"role": "user", "content": "Who are you? Just answer the question."},
]
pipe = pipeline("text-generation", model=model_name, device=0)
answer = pipe(messages, max_new_tokens=10)
print(answer[0]["generated_text"][1]["content"])

2024-10-30 15:07:04.383832: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 15:07:04.422881: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 15:07:04.422912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 15:07:04.422933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 15:07:04.430997: I tensorflow/core/platform/cpu_feature_g

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


I am Qwen, a large language model created


In [3]:
# create empty response list which is transformed to a dataframe

response_list = []

# define rounds to be tested
rounds = 25

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
          answer = pipe(messages, max_length=1000, max_new_tokens=10)
          response = answer[0]["generated_text"][1]["content"]
          print(response)
          
          df_response_input = [round,question_ID, question, answer_options, k,  response]
          response_list.append(df_response_input)
          torch.cuda.empty_cache()
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
              answer = pipe(messages, max_length=1000, max_new_tokens=10)
              response = answer[0]["generated_text"][1]["content"]
              print(response)
              df_response_input = [round,question_ID, question, answer_options, n, response]
              response_list.append(df_response_input)
              torch.cuda.empty_cache()

Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


1


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

11=Essential feature


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


6


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


11=Essential feature
1=Not an essential feature


In [4]:
print("Length of responses: ", len(response_list))

Length of responses:  1550


In [5]:
# create a bias interaction experiment where there is a question-side and answer option-side bias
# here we use
perturbed_question = "list_answer_options_reversed"
perturbed_answer = "question_paraphrased"

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      # first retrieve original question
      question = questionnaire.iloc[row]["question_paraphrased"]
      # second retrieve bias perturbed answer options from the questionnaire  
      answer_options = questionnaire.iloc[row]["list_answer_options_reversed"]      
      messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]    
      answer = pipe(messages, max_length=1000, max_new_tokens=10)       
      response = answer[0]["generated_text"][1]["content"]         
      print(response)  
      df_response_input = [round,question_ID, question, answer_options, "interaction_paraphrase_reversed",  response]
      response_list.append(df_response_input)
      torch.cuda.empty_cache()

Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


1=Very important
1


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

5


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5


Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


10=Essential feature
5


In [6]:
print("Length of responses after interaction experiments: ", len(response_list))

Length of responses after interaction experiments:  3100


In [7]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "Gwen2.5-7B-Instruct"

In [8]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/final_interviews/{current_date}_Gwen2.5-7B-Instruct_selected_interviews.csv", encoding="utf-8", sep=";")